# Detección de sombras 

*: El "ejemplo nubes" sale de acá: https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

## Autenticación

In [39]:
import os
import ee
from pprint import pprint
from google.auth.transport.requests import AuthorizedSession


##################################################################
# CREDENTIALS =======
print('\nEnviando credenciales a la nube ...')

PROJECT = 'pruebas-gee-00' # Ej: pruebas-engine-00
# SERVICE_ACCOUNT = 'pruebas-gee-jmb@pruebas-gee-00.iam.gserviceaccount.com'
SERVICE_ACCOUNT = 'jumanbar@gmail.com'
KEY = os.path.join(os.getcwd(), 'debian-key.json')
rest_api_url = 'https://earthengine.googleapis.com/v1beta/projects/{}/table:computeFeatures'

print('\LLAVE:', KEY)

# PROBLEMAS ACA / INICIO
credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY)
scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform'])

session = AuthorizedSession(scoped_credentials)
ee.Initialize(credentials)
# PROBLEMAS ACA / FIN
###################################################################


Enviando credenciales a la nube ...
\LLAVE: c:\Users\juan.barreneche\Documents\gee_automatizacion\debian-key.json


## Funciones


### Variables y tal

In [40]:
"""
# VARIABLES, VALORES INICIALES
# Definir estas cosas para que el VS code no ponga
# alertas todo el tiempo
id_zona = 1
zona = 'PALMAR'
geom = ee.FeatureCollection('users/brunogda/zonas_palmar_represa_dis')\
    .first()\
    .geometry()

p = [10, 50, 90]
cloud_perc = 25
cloud_perc2 = 25
MAX_CLOUD_PROBABILITY = 10

ini_date = '2023-03-01'
end_date = '2023-03-31'
mask_ndwi = ee.Image(0)
lo_ndwi = ee.Image(0)

# Para sombras
BUFFER = 50
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.5
CLD_PRJ_DIST = 1
"""

from inifun import *
import calfun as cf
id_zona = 5
nzonas = 60
isValidZoneID(id_zona, nzonas)
asset_string = getAssetFromIdZona60(id_zona)
print(f'Asset string: {asset_string}')

# Polígono:
geom = ee.FeatureCollection(asset_string).first().geometry()

p = [10, 50, 90]
cloud_perc  = 50
cloud_perc2 = 50
# MAX_CLOUD_PROBABILITY = 10
MAX_CLOUD_PROBABILITY = 30

ini_date = '2023-06-17'
ini_date_o3 = '2023-05-17'
end_date = '2023-06-18'


# Cloud shadows
BUFFER = 50
CLD_PRB_THRESH = 50
CLD_PRJ_DIST = 1

# COPIAR VARIABLES AL NAMESPACE DE calfun:
cf.zona = 'RN60Z'
cf.id_zona = id_zona
cf.geom = geom
cf.p = p
cf.cloud_perc = cloud_perc
cf.cloud_perc2 = cloud_perc2
cf.MAX_CLOUD_PROBABILITY = MAX_CLOUD_PROBABILITY
cf.end_date = end_date
cf.ini_date = ini_date
cf.ini_date_o3 = ini_date_o3

# DEFINIR PROPIEDAD Y VALOR PARA FILTRAR S2:
prp = 'MGRS_TILE'
val = '21HVD'

if nzonas == 7:
    if id_zona >= 4 and id_zona < 6:
        prp = 'SENSING_ORBIT_NUMBER'
        val = 124
    if id_zona >= 6:
        val = '21HWD'
else:
    if id_zona >= 1 and id_zona < 18:
        val = '21HVD'

    if id_zona == 18:
        val = '21HWD'

    if id_zona >= 19 and id_zona < 37:
        prp = 'SENSING_ORBIT_NUMBER'
        val = 124

    if id_zona >= 37 and id_zona < 56:
        val = '21HWD'

    if id_zona >= 56 and id_zona < 59:
        val = '21HXD'

    if id_zona >= 59 and id_zona <= 60:
        val = '21HXE'

Asset string: users/brunogda/RN60/05_rn


### Funciones nuevas

In [41]:
###################################################################
# PARA GESTIONAR SOMBRAS DE NUBES
# Fuente:
# https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

def bandClouds(img):
    # Basada en maskClouds (ver Actuales)
    cld_prb = ee.Image(img.get('cloud_mask')).select('probability')
    is_cloud = cld_prb.gte(MAX_CLOUD_PROBABILITY).rename('clouds')
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_cloud_bands(img):
    cld_prb = ee.Image(img.get('cloud_mask')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):

    # i. Identify dark NIR pixels that are not water (potential cloud shadow
    # pixels).

    SR_BAND_SCALE = 1e4

    ## OPCION 1: S2_R2
    ## Identify water pixels from the SCL band. (S2_SR)
    # not_water = img.select('SCL').neq(6)
    # dark_pixels = img.select('B8').lt(NIR_DRK_THRESH * SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    # dark_pixels = img.select('B8').lt(NIR_DRK_THRESH * SR_BAND_SCALE).rename('dark_pixels')

    # ## OPCION 2: *_HARMONIZED
    # ## Usar lo_ndwi (valores bajos de NDWI).

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds')
        .directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST * 10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # dark_pixels = img.select('B8').lt(NIR_DRK_THRESH * SR_BAND_SCALE).multiply(hi_ndwi.Not()).rename('dark_pixels')
    dark_pixels = (img.select('B8')
      .lt(NIR_DRK_THRESH * SR_BAND_SCALE)
      .multiply(hi_ndwi)
      .rename('dark_pixels'))

    # dark_pixels = img.select('B8').lt(NIR_DRK_THRESH * SR_BAND_SCALE).rename('dark_pixels')
    # dark_pixels = img.select('B8').lt(NIR_DRK_THRESH * SR_BAND_SCALE).multiply(ndwi.select('nd').lt(0.5)).rename('dark_pixels')

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = bandClouds(img)
    # img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER * 2 / 20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    # return img_cloud_shadow.addBands(is_cld_shdw)
    return img_cloud_shadow.updateMask(is_cld_shdw)

###################################################################

## Image Collections

In [42]:
MSI_source = 'COPERNICUS/S2_HARMONIZED'    # Nuestro, original
# MSI_source = 'COPERNICUS/S2_SR'            # Ejemplo nubes*
# MSI_source = 'COPERNICUS/S2_SR_HARMONIZED' # Otra prueba...

# Sentinel multibanda
MSI = ee.ImageCollection(MSI_source)

# Probabilidad de nubes:
S2_clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

## UMBRAL DE NIR

> Original: 0.15

Esto determina cuáles se consideran como Dark Pixels... el tema bravo es que al ser sobre superficie del agua, todos tienen valores muy bajos de NIR.

En las pruebas que hice, un rango de umbrales bastante pequeño (0.029-0.040), el cambio es muy grande, lo cual es mala señal. Ideal es una distribución que distinga claramente entre sombra sobre el agua y agua sin sombra.

¿Hay alguna otra forma de intentar separar ambas situaciones?

In [43]:
NIR_DRK_THRESH = 0.038 # Original: 0.15

In [46]:
print(ini_date)

2023-06-17


## Run



In [70]:
def eeToFoliumPoly(poly: ee.geometry.Geometry) -> list:
    coor = poly.getInfo()['coordinates'][0]
    out = map(lambda xy: [xy[1], xy[0]], coor)
    return(list(out))

buffer_example = geom.buffer(500)

import folium
m = folium.Map(location=[-33.041, -57.374], zoom_start=13, tiles = 'OpenStreetMap')

folium.Polygon(locations=eeToFoliumPoly(buffer_example),
               fill_opacity=0.5,
               fill_color="blue",
               weight=0,
               popup='Bonete'
               ).add_to(m)

folium.Polygon(locations=eeToFoliumPoly(geom),
               fill_opacity=0.5,
               fill_color="red",
               weight=0,
               popup='Bonete'
               ).add_to(m)
m

### S5Clouds

In [83]:
geom_buffer = geom.buffer(10000)

S5cloud = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CLOUD')\
    .filterDate(f'{ini_date}T00:00:00', f'{ini_date}T23:59:59')
    # .map(function(image) { return image.clip(geometry) })

cbh = S5cloud.select('cloud_base_height')
# pprint(cbh.getInfo())
stats = cbh.mean().reduceRegion(
  reducer = ee.Reducer.median(),
  geometry = geom_buffer,
  scale = 1113.2  # metros
)
print('Stats: ', stats.getInfo())


# var reducer = ee.Reducer.median().combine;
reducer = ee.Reducer.median().combine(
  reducer2 = ee.Reducer.percentile([10, 50, 60, 70, 75, 80, 85, 90]),
  sharedInputs = True
)

reducer = reducer.combine(
  reducer2 = ee.Reducer.min(),
  sharedInputs = True
)

reducer = reducer.combine(
  reducer2 = ee.Reducer.max(),
  sharedInputs = True
)

multiStats = cbh.mean().log10().reduceRegion(
  reducer = reducer,
  geometry = geom_buffer,
  scale = 1113.2
)
print('===== Multistats:')
pprint(multiStats.getInfo());


Stats:  {'cloud_base_height': 47.2971760085512}
===== Multistats:
{'cloud_base_height_max': 1.8513582771966435,
 'cloud_base_height_median': 1.6745930675610232,
 'cloud_base_height_min': 1.6290781358087958,
 'cloud_base_height_p10': 1.6290781568417618,
 'cloud_base_height_p50': 1.6745930675610232,
 'cloud_base_height_p60': 1.6795917197682464,
 'cloud_base_height_p70': 1.6795917197682464,
 'cloud_base_height_p75': 1.6948243131194511,
 'cloud_base_height_p80': 1.6987492587554667,
 'cloud_base_height_p85': 1.6987492587554667,
 'cloud_base_height_p90': 1.6987492587554667}


In [ ]:




# Filter input collections by desired data range and region.
criteria = ee.Filter.And(ee.Filter.bounds(geom),
                         ee.Filter.date(ini_date, end_date))

S2_clouds = S2_clouds.filter(criteria)

# Esto se fija en qué imágenes de la colección hay pocas nubes en total y se
# queda con esas:
S2_mask = MSI.filter(criteria)\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', "less_than", cloud_perc2)

# FILTER Sentinel 2 collection
FC2 = S2_mask.filter(ee.Filter.eq(prp, val)).map(cf.maskEdges)

## NDWI
## Sirve con S2_HARMONIZED y S2_SR_HARMONIZED
ndwi = S2_mask.median().normalizedDifference(['B3', 'B8'])
hi_ndwi = ndwi.select('nd').gte(0.2)
lo_ndwi = ndwi.select('nd').lt(0.2)
mask_ndwi = hi_ndwi.updateMask(hi_ndwi)

# FILTER Sentinel 2 collection
# FC2 = MSI.filterDate(ini_date, end_date)\
#     .filterBounds(geom)\
#     .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', "less_than", cloud_perc)\
#     .filter(ee.Filter.eq(prp, val))

# Join S2 SR with cloud probability dataset to add cloud mask.
FC2_with_cloud_mask = ee.ImageCollection(
    ee.Join.saveFirst('cloud_mask').apply(
        primary = FC2,
        secondary = S2_clouds,
        condition = ee.Filter.equals(
            leftField = 'system:index',
            rightField = 'system:index')
        )
    )

S2_cloud_masked = FC2_with_cloud_mask.map(add_cld_shdw_mask)
# pprint.pprint(S2_cloud_masked.getInfo())



### Para ver las bandas del FC2_with_cloud_mask:

In [45]:
# pprint.pprint(FC2_with_cloud_mask.getInfo()['features'][0]['bands'])
# ! C:/Users/juan.barreneche/AppData/Local/Programs/Python/Programs/Python/Python310/python.exe -m pip install geemap

## Viz

### Proyección nubes

Esta parte es extraída de una de las funciones (ver [Nuevas](#scrollTo=Nuevas)). Lo puse acá porque quería ver cómo funcionaba eso de la proyección y la función `directionalDistanceTransform`

In [29]:
img_cloud = bandClouds(FC2_with_cloud_mask.first())

# Determine the direction to project cloud shadow from clouds (assumes UTM projection).
shadow_azimuth = ee.Number(90).subtract(ee.Number(img_cloud.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

# Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
cld_proj = (img_cloud.select('clouds')
    .directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST * 100)
    .reproject(**{'crs': img_cloud.select(0).projection(), 'scale': 10})
    .select('distance'))

In [30]:
pprint(FC2.getInfo())

{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:32721',
                          'crs_transform': [60, 0, 399960, 0, -60, 6400000],
                          'data_type': {'max': 65535,
                                        'min': 0,
                                        'precision': 'int',
                                        'type': 'PixelType'},
                          'dimensions': [1830, 1830],
                          'id': 'B1'},
                         {'crs': 'EPSG:32721',
                          'crs_transform': [10, 0, 399960, 0, -10, 6400000],
                          'data_type': {'max': 65535,
                                        'min': 0,
                                        'precision': 'int',
                                        'type': 'PixelType'},
                          'dimensions': [10980, 10980],
                          'id': 'B2'},
                         {'crs': 'EPSG:32721',
                          'crs_transform': [10, 0, 

In [37]:
# pprint(S2_clouds.getInfo())
# pprint(FC2.getInfo())
from math import tan, pi, log10
import numpy as np
ze = FC2.getInfo()['features'][0]['properties']['MEAN_SOLAR_ZENITH_ANGLE']
print(f"Mean solar zenith angle:\t{ze:.2f}")
d_nubes = 400
print(f"Distancia obs. de las nubes:\t{d_nubes:.2f}")
al = 90 - ze
h = d_nubes * tan(pi * al / 180)
print("Altura de una nube ejemplo:\t{}m (log10: {})".format(np.around(h, decimals=1), np.around(log10(h), decimals=3)))

# Si h = 500
h_ej = 500
d_nubes = h_ej / tan(pi * al / 180)
print(f"Distancia de sombra a nube (altura: {h_ej}m), hipotética:\t{d_nubes:.2f}m ({d_nubes/10:.2f} px)")

# Si h = 50
h_ej = 50
d_nubes = h_ej / tan(pi * al / 180)
print(f"Distancia de sombra a nube (altura: {h_ej}m), hipotética:\t{d_nubes:.2f}m ({d_nubes/10:.2f} px)")


Mean solar zenith angle:	62.12
Distancia obs. de las nubes:	400.00
Altura de una nube ejemplo:	211.6m (log10: 2.326)
Distancia de sombra a nube (altura: 500m), hipotética:	945.01m (94.50 px)
Distancia de sombra a nube (altura: 50m), hipotética:	94.50m (9.45 px)


Posible estimación de alturas de nubes: https://code.earthengine.google.com/448f9425387963431ec1ac567b206bc5; (https://code.earthengine.google.com/?scriptPath=users%2Fjumanbar%2FpruebasRN%3AAlturaNubes)

### Armar el mapa

Acá simplemente es agregar y probar capas para visualizar. Atento a la función `.selfMask` que ayudó a que se vea lo deseado.

In [32]:
import geemap
m = geemap.Map(center=(-33.09068, -57.30898), zoom=12)
m.addLayer(FC2_with_cloud_mask.first(), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'FC2 w/cloud mask', False, 1)
m.addLayer(FC2.first(), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'FC2', True, 1)

m.addLayer(S2_clouds.first().select('probability'), {'min': 0, 'max': 100, 'palette': ['skyblue', 'white']}, 'S2_clouds', True, 1)

# m.addLayer(FC2.first(), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'FC2', False, 1)
m.addLayer(hi_ndwi, {'min': 0, 'max': 1, 'palette': ['Chocolate', 'blue']}, 'Water (hi_ndwi)', False, 1)
m.addLayer(ndwi.select('nd').gte(0.4), {'min': 0, 'max': 1, 'palette': ['Chocolate', 'blue']}, 'Water (ndwi.select("nd").gte(0.4))', False, 1)
# m.addLayer(lo_ndwi, {'min': 0, 'max': 1, 'palette': ['#b3ecff', '#b32400']}, 'lo_ndwi', True, 1, 8)

m.addLayer(cld_proj, {'min': 0, 'max': 20, 'palette': ['white', 'red']}, 'DISTANCE', True, 1)
m.addLayer(S2_cloud_masked.first().select('clouds').selfMask(), {'min': 0, 'max': 1, 'palette': ['skyblue', 'white']}, 'Clouds', False, 1)

# m.addLayer(S2_cloud_masked.first().select('clouds'), {'palette': 'e056fd'}, 'Clouds 2', True, 1)
# m.addLayer(S2_clouds.first().select('probability'), {'min': 0, 'max': 100, 'palette': ['skyblue', 'white']}, 'P(Clouds)', False, 1)
# m.addLayer(S2_cloud_masked.first(), {'bands': ['clouds'], 'palette': ['yellow']}, 'Clouds', True, 1)

m.addLayer(S2_cloud_masked.first().selfMask(), {'bands': ['cloud_transform'], 'palette': ['orange']}, 'Cloud transform', False)
m.addLayer(S2_cloud_masked.first().select('dark_pixels').selfMask(), {'min': 0, 'max': 1, 'palette': ['white', 'purple']}, 'Dark pixels mask', False, 0.8)
# m.addLayer(S2_cloud_masked.first().select('dark_pixels'), {'min': 0, 'max': 1, 'palette': ['white', 'purple']}, 'Dark pixels', False)
m.addLayer(S2_cloud_masked.first().selfMask(), {'bands': ['shadows'], 'palette': ['black']}, 'Shadows', False)
m.addLayer(FC2_with_cloud_mask.first().select('B8'), {'min': 0, 'max': 1000, 'palette': ['MidnightBlue', 'red']}, 'NIR', False)
# m.addLayerControl()

display(m)


Map(center=[-33.09068, -57.30898], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

Como se puede ver, las sombras son complicadas...